<a href="https://colab.research.google.com/github/Hemanthraju123/akaike-assign/blob/main/News_Sentiment_Analysis_TTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install beautifulsoup4 newspaper3k transformers torch googletrans==3.1.0a0 gTTS requests pandas streamlit


  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 34.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 976.2 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.3 MB/s eta 0:00:00
  

In [2]:
!pip install lxml[html_clean]

In [3]:
import newspaper
from newspaper import Article
from bs4 import BeautifulSoup
import requests
from transformers import pipeline
from googletrans import Translator
from gtts import gTTS
import pandas as pd
import json
from IPython.display import Audio


In [4]:
def fetch_news(company_name):
    # Use News API to get articles
    api_key = "d97ab9d4a6424a9fb3b144269889c393"  # Get from https://newsapi.org
    url = f"https://newsapi.org/v2/everything?q={company_name}&apiKey={api_key}&language=en"

    response = requests.get(url)
    articles = response.json()['articles'][:10]

    processed_articles = []
    for article in articles:
        try:
            # Extract article using newspaper3k
            news_article = Article(article['url'])
            news_article.download()
            news_article.parse()
            news_article.nlp()  # Generates summary

            processed_articles.append({
                "title": news_article.title,
                "summary": news_article.summary,
                "url": article['url'],
                "keywords": news_article.keywords
            })
        except:
            continue

    return processed_articles

In [5]:
def analyze_sentiment(articles):
    sentiment_analyzer = pipeline("sentiment-analysis")

    for article in articles:
        result = sentiment_analyzer(article['summary'])[0]
        article['sentiment'] = {
            'label': result['label'],
            'score': result['score']
        }

    return articles


In [6]:
def comparative_analysis(articles):
    sentiment_counts = {
        'POSITIVE': 0,
        'NEGATIVE': 0,
        'NEUTRAL': 0
    }

    topics = {}
    for article in articles:
        sentiment_counts[article['sentiment']['label']] += 1
        for keyword in article['keywords']:
            topics[keyword] = topics.get(keyword, 0) + 1

    return {
        'sentiment_distribution': sentiment_counts,
        'common_topics': dict(sorted(topics.items(), key=lambda x: x[1], reverse=True)[:5])
    }

In [7]:
def generate_hindi_audio(text):
    translator = Translator()
    hindi_text = translator.translate(text, dest='hi').text

    tts = gTTS(text=hindi_text, lang='hi')
    tts.save('summary.mp3')

    return Audio('summary.mp3')


In [8]:
def process_company_news(company_name):
    # Fetch and process news
    articles = fetch_news(company_name)
    articles_with_sentiment = analyze_sentiment(articles)
    analysis = comparative_analysis(articles_with_sentiment)

    # Generate summary
    summary = f"Analysis for {company_name}:\n"
    summary += f"Total articles analyzed: {len(articles)}\n"
    summary += f"Sentiment distribution: {analysis['sentiment_distribution']}\n"
    summary += f"Main topics: {', '.join(analysis['common_topics'].keys())}"

    # Generate audio
    audio = generate_hindi_audio(summary)

    return {
        'articles': articles_with_sentiment,
        'analysis': analysis,
        'summary': summary,
        'audio': audio
    }


In [9]:
from IPython.display import display, HTML

company_name = input("Enter company name: ")
results = process_company_news(company_name)

# Display results
display(HTML(f"<h2>Results for {company_name}</h2>"))
print("\nSummary:")
print(results['summary'])
print("\nAudio Summary (in Hindi):")
display(results['audio'])

Enter company name: apple


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu



Summary:
Analysis for apple:
Total articles analyzed: 0
Sentiment distribution: {'POSITIVE': 0, 'NEGATIVE': 0, 'NEUTRAL': 0}
Main topics: 

Audio Summary (in Hindi):
